Задание 1

Реализуйте обучение нейронной сети из одного нейрона для задачи предсказания стоимости квартир boston house prices или california housing prices с использованием pytorch.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

house = pd.read_csv('housing.csv', header=None, names=column_names, delimiter=r'\s+')
house.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [ ]:
features = house.iloc[:, :-1]
target = house.iloc[:, -1]

In [ ]:
sc = StandardScaler()
features = sc.fit_transform(features)
target = target.to_numpy(target)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, train_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(101, 13) (405, 13) (101,) (405,)


In [ ]:
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

In [ ]:
batch_size = 10

In [ ]:
dataset = TensorDataset(X_train, y_train)
data_iter = DataLoader(dataset, batch_size, shuffle=True)

In [ ]:
model = torch.nn.Sequential(torch.nn.Linear(X_train.shape[1], 1))
loss = torch.nn.MSELoss(reduction='mean')
trainer = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 100
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:

        X = X.to(torch.float32)
        y = y.to(torch.float32)

        trainer.zero_grad()

        l = loss(model(X).reshape(-1), y)

        l.backward()
        
        trainer.step()
    l = loss(model(X_test.to(torch.float32)).reshape(-1), y_test.to(torch.float32))
    if epoch % 5 == 0:
        print(f'Epoch: {epoch}')
        print(f'loss: {l.item()}')
        print(f'weight: {model[0].weight.data}')
        print(f'bias: {model[0].bias.data}')
        print('-'*30)

Epoch: 5
loss: 469.1436767578125
weight: tensor([[-0.5716, -0.1468, -0.0028,  0.2759, -0.2680,  0.5581, -0.2191, -0.1062,
         -0.4275, -0.3355, -0.5003,  0.2384, -0.6879]])
bias: tensor([2.2544])
------------------------------
Epoch: 10
loss: 378.2428894042969
weight: tensor([[-0.8231, -0.1470, -0.0969,  0.3140, -0.3255,  0.8906, -0.4505, -0.0730,
         -0.6156, -0.3964, -0.8779,  0.4167, -1.2067]])
bias: tensor([4.3209])
------------------------------
Epoch: 15
loss: 305.8080749511719
weight: tensor([[-1.0007, -0.1989, -0.0425,  0.4302, -0.2497,  1.3208, -0.5161, -0.1975,
         -0.6826, -0.3960, -1.3036,  0.5326, -1.6856]])
bias: tensor([6.2238])
------------------------------
Epoch: 20
loss: 252.1744842529297
weight: tensor([[-1.1434, -0.1974, -0.0286,  0.4616, -0.0973,  1.5027, -0.5740, -0.3094,
         -0.7425, -0.3529, -1.6064,  0.5785, -2.0114]])
bias: tensor([7.8731])
------------------------------
Epoch: 25
loss: 208.03211975097656
weight: tensor([[-1.2557, -0.1967,

Задание 2 (дополнительное и необязательное)

Реализуйте наивный байесовский классификатор для MNIST (можно взять всего две цифры “1” и “2”)
Сравните качество с качеством реализации из библиотеки scikit-learn (naive_bayes).


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
import torchvision as tv
from sklearn.metrics import accuracy_score

In [ ]:
train_ds = tv.datasets.MNIST('.',
                             train=True,
                             transform=tv.transforms.ToTensor(),
                             download=True)

test_ds = tv.datasets.MNIST('.',
                            train=False,
                            transform=tv.transforms.ToTensor(),
                            download=True)

In [ ]:
indices_train = (train_ds.targets == 1) | (train_ds.targets == 2)
indices_test = (test_ds.targets == 1) | (test_ds.targets == 2)

In [ ]:
train_ds.data, train_ds.targets = train_ds.data[indices_train], train_ds.targets[indices_train]
test_ds.data, test_ds.targets = test_ds.data[indices_test], test_ds.targets[indices_test]

In [ ]:
train_ds.data.shape, train_ds.targets.shape, test_ds.data.shape, test_ds.targets.shape

(torch.Size([12700, 28, 28]),
 torch.Size([12700]),
 torch.Size([2167, 28, 28]),
 torch.Size([2167]))

In [ ]:
train_data = train_ds.data.reshape(train_ds.data.shape[0],-1)
test_data = test_ds.data.reshape(test_ds.data.shape[0],-1)

In [ ]:
X_train = train_data.numpy()
y_train = train_ds.targets.numpy()
X_test = test_data.numpy()
y_test = test_ds.targets.numpy()

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((12700, 784), (12700,), (2167, 784), (2167,))

In [ ]:
clf = GaussianNB()
clf.fit(X_train, y_train)
print(accuracy_score(y_test, clf.predict(X_test)))

0.9616982002768805


In [ ]:
class NB:

  def fit(self, X, y):
    n_examples, n_features = X.shape
    self.classes = np.unique(y)
    n_classes = len(self.classes)

    self.mean = np.zeros((n_classes, n_features))
    self.var = np.zeros((n_classes, n_features))
    self.prior = np.zeros(n_classes)

    for id, cla in enumerate(self.classes):
      X_c = X[y == cla]
      self.mean[id, :] = X_c.mean(axis=0)
      self.var[id, :] = X_c.var(axis=0)
      self.prior[id] = X_c.shape[0] / float(n_examples)
  
  def predict(self, X):
    y_pred = [self._predict(x) for x in X]
    return np.array(y_pred)

  def _predict(self, x):
    posteriors = []

    for id, c in enumerate(self.classes):
      prior = np.log(self.prior[id])
      posterior = np.sum(np.log(self._pdf(id, x)))
      posterior = posterior + prior
      posteriors.append(posterior)
    return self.classes[np.argmax(posteriors)]

  def _pdf(self, cla_id, x):
    mean = self.mean[cla_id]
    var = self.var[cla_id]
    numerator = np.exp(-((x - mean) ** 2) / (2 * var))
    denom = np.sqrt(2 * np.pi * var)
    return numerator / denom

In [ ]:
cl = NB()
cl.fit(X_train, y_train)
print(accuracy_score(y_test, cl.predict(X_test)))

0.5237655745269958


<ipython-input-374-6d679fe350f2>:35: RuntimeWarning: invalid value encountered in true_divide
  numerator = np.exp(-((x - mean) ** 2) / (2 * var))
<ipython-input-374-6d679fe350f2>:27: RuntimeWarning: divide by zero encountered in log
  posterior = np.sum(np.log(self._pdf(id, x)))
<ipython-input-374-6d679fe350f2>:35: RuntimeWarning: divide by zero encountered in true_divide
  numerator = np.exp(-((x - mean) ** 2) / (2 * var))
<ipython-input-374-6d679fe350f2>:37: RuntimeWarning: invalid value encountered in true_divide
  return numerator / denom
